# Testing Balanced Autoencoder on Old TL model 
- Weighted Accuracy: 0.641255253
- Sensitivity/Recall: 0.601520926
- Ephochs 100,300

In [1]:
import os
import sys
import importlib
import pickle
import keras
import pdb
import pandas as pd
import numpy as np
from datetime import date, datetime

2023-08-24 11:00:30.882635: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 11:00:30.933352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 11:00:30.934226: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 11:00:31.876353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
execution_path = "../../"
import os
data_ndarr = np.load(r"all_pids_q=0.3.npy", allow_pickle=True)
datadict = data_ndarr.item()

In [3]:
from TL_Packages import *

# Architecture_GROUP = "TL"
Architecture_GROUP = "model1"

def runtime():
    print("Runtime:", datetime.now().hour, "o' clock and", datetime.now().minute, "minutes")

def run_tlsi(file):
    """
        run transfer learning algorithm on sample imbalanced data set
    """
    pass

In [4]:
p_ids = datadict.keys()
smaple_size_dict={}
print("patients: n =", len(p_ids), end='\n\n')

for p_id in p_ids:
    df = datadict[p_id]
    df.drop(['id','date','am_pef_org','BMI','sex', 'age'], axis=1, inplace=True) 
    df = np.asarray(df).astype(np.float32)
    print(p_id, "shape:", df.shape)#, end="\t\t")
    smaple_size_dict[p_id]=df.shape[0]

datadict[p_id].head(4)

patients: n = 26

SB-001 shape: (188, 28)
SB-003 shape: (180, 28)
SB-008 shape: (178, 28)
SB-011 shape: (210, 28)
SB-012 shape: (194, 28)
SB-022 shape: (204, 28)
SB-023 shape: (194, 28)
SB-025 shape: (117, 28)
SB-028 shape: (92, 28)
SB-033 shape: (53, 28)
SB-037 shape: (210, 28)
SB-043 shape: (191, 28)
SB-056 shape: (191, 28)
SB-059 shape: (203, 28)
SB-060 shape: (88, 28)
SB-070 shape: (202, 28)
SB-071 shape: (124, 28)
SB-073 shape: (161, 28)
SB-078 shape: (153, 28)
SB-079 shape: (142, 28)
SB-080 shape: (181, 28)
SB-081 shape: (34, 28)
SB-082 shape: (137, 28)
SB-083 shape: (206, 28)
SB-089 shape: (49, 28)
SB-112 shape: (180, 28)


,income,road_dist,cooking,y_am_pef,tempin,humidin,pm25in,co2in,tempdiffin,humidiffin,...,windsd,humid,varp,dewpt,airp,seap,solrhr,solramnt,grdt,class
1,0.857,0.0,0.167,0.558,0.216,0.513,0.358,0.678,0.562,0.364,...,0.107,0.668,0.174,0.471,0.822,0.819,0.165,0.003,0.206,0.0
2,0.857,0.0,0.167,0.006,0.299,0.521,0.411,0.700,0.436,0.672,...,0.171,0.388,0.190,0.491,0.676,0.670,0.603,0.165,0.383,0.0
3,0.857,0.0,0.167,0.078,0.420,0.713,0.372,0.918,0.469,0.821,...,0.515,0.359,0.197,0.481,0.519,0.518,0.485,0.104,0.346,1.0
4,0.857,0.0,0.167,0.428,0.438,0.468,0.224,0.762,0.560,0.490,...,0.338,0.527,0.175,0.467,0.596,0.596,0.491,0.141,0.339,0.0


#### dropped columns

`date` was dropped above, because it would throw `ValueError: could not convert string to float: '2017-11-01'`
`id` and `am_pef_org` were dropped because neither are explanatory variables.

(Note: `am_pef_org` is the UNSHIFTED, NON-NORMALIZED value for a person's morning exposure. It is relatively raw, and should almost exclusively be used for classification purposes)

In [5]:
target_grp = ["SB-001", "SB-003", "SB-008", "SB-011", "SB-012", "SB-022", "SB-023", "SB-025", "SB-028", "SB-037", "SB-043", "SB-056", "SB-059", "SB-060", "SB-070", "SB-071", "SB-073", "SB-078", "SB-079", "SB-080", "SB-082", "SB-083", "SB-112"]

print(len(target_grp))

23


In [6]:
from TL_Packages import *

# Architecture_GROUP = "TL"
Architecture_GROUP = "model"
RESULT_DIR = 'Results/'
EVALUATION_METRICS = ["Weighted Accuracy", "Sensitivity/Recall", "Specificity", "Precision_class0", "Precision_class1", "Precision_avg", "F1_class0", "F1_class1", "F1_avg", "auc_roc_score", "False_Discovery_Rate", "False_Negative_Rate", "False_Omission_Rate", "False_Positive_Rate", "Jaccard"]

# TL_Base_Model

In [7]:
from tqdm import tqdm
import os.path

def TL_Base_Model(POPULATION_EPOCH_SIZE,PATIENT_EPOCH_SIZE, p_ids):
    df_evaluation_results = pd.DataFrame(columns=EVALUATION_METRICS)

    minority_variable=0
    #Target MOdel dataframe
    df_evaluation_results = pd.DataFrame(columns=EVALUATION_METRICS)
    #Population model results
    minor_var=minority_variable

    for target in tqdm(p_ids):  
        if target not in target_grp:
            continue
        runtime()
        print("~~~~~~~~~~~", POPULATION_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", PATIENT_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", target, "~~~~~~~~~~~~~~~~~~~")

        patient_dir = RESULT_DIR + "/diagrams/target"

        # create directory for this patient
        if not (patient_dir and patient_dir):
            os.makedirs(patient_dir)

        CLASS_VAR = 'class'
        # load population data
        X_train, X_test, Y_train, Y_test = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var,test_frac=0.05,
                                                      transfer=True, balance=True)

        # pre-train model with population data
        net = network_tl(X_train, Y_train, n_hidden=32, learning_rate=0.001, device='cpu')
        net.make_pop()

        net.train(n_iters=POPULATION_EPOCH_SIZE)

        VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}'
        #save model
        modelname = f"{target}-{VERSION_TAG}.hdf5"
        model_path= patient_dir + "/" + modelname
        net.nn_pop.save(model_path) #nn_pop

        #save training curve
        training_curve_file = patient_dir +"/"+ f'{VERSION_TAG}_training_curve_{target}'
        net.save_training_curve(training_curve_file)
        # load target patient data
        dataset_list = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var, #explanatory_var
                                balance=True)

        kfold_evaluation_results_list = []
        kfold_confusion_matrix_list = []

        for i in range(3):
            # load the target data in the current round
            X_train, X_test, Y_train, Y_test = dataset_list[i]
            net.make_ind()
            net.retrain(X_train, Y_train, n_iters=PATIENT_EPOCH_SIZE)
            print("done RE-training", i, "of 3", end=". ")

            # test
            VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_{i}'
            #save model
            modelname = f"{target}-{VERSION_TAG}.hdf5"
            model_path= patient_dir + "/" + modelname
            net.nn_ind.save(model_path) #nn_pop

            #save retraining curve
            training_curve_file = patient_dir +"/"+ f'{VERSION_TAG}_training_curve_{target}'
            net.save_training_curve(training_curve_file)

            results, c_matrix = net.test(X_test, Y_test)
            kfold_evaluation_results_list += [results]
            kfold_confusion_matrix_list += [c_matrix]

            print("done testing", i, "of 3.")

        VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}'

        # save average of evaluation results
        confusion_matrixes_file = (f"{str(patient_dir)}/{VERSION_TAG}_confusion_matrix_{target}")

        avg_results = pd.DataFrame(kfold_evaluation_results_list, columns=EVALUATION_METRICS).mean().values

        avg_results = avg_results.reshape(1, 15)

        df = pd.DataFrame(avg_results, columns=EVALUATION_METRICS)
        df.index = [f"{target}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}"]
        df_evaluation_results = pd.concat([df_evaluation_results,df], axis = 0)

        net.save_confusion_matrix(kfold_confusion_matrix_list, confusion_matrixes_file)

    # output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics_{sorted(set(p_ids).difference(target_grp))[0]}.csv')
    output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics.csv')

    if os.path.isfile(output_path):
        df_evaluation_results.to_csv(output_path,
                                        mode='a',
                                        header=False
                                    )
    else:
                df_evaluation_results.to_csv(output_path,
                                        mode='a'
                                    )

    print("evaluation results saved for", VERSION_TAG)

In [8]:
POPULATION_EPOCH_SIZE = [*range(500, 1600, 500)]
PATIENT_EPOCH_SIZE = [*range(500, 1600, 500)]

print("POPULATION_EPOCH_SIZE:",POPULATION_EPOCH_SIZE, "Len:", len(POPULATION_EPOCH_SIZE))
print("PATIENT_EPOCH_SIZE:",PATIENT_EPOCH_SIZE, "Len:", len(PATIENT_EPOCH_SIZE))
print("target_grp:",target_grp, "Len:", len(target_grp))
print("Total records in each file:",len(POPULATION_EPOCH_SIZE) * len(PATIENT_EPOCH_SIZE) * len(target_grp))

POPULATION_EPOCH_SIZE: [500, 1000, 1500] Len: 3
PATIENT_EPOCH_SIZE: [500, 1000, 1500] Len: 3
target_grp: ['SB-001', 'SB-003', 'SB-008', 'SB-011', 'SB-012', 'SB-022', 'SB-023', 'SB-025', 'SB-028', 'SB-037', 'SB-043', 'SB-056', 'SB-059', 'SB-060', 'SB-070', 'SB-071', 'SB-073', 'SB-078', 'SB-079', 'SB-080', 'SB-082', 'SB-083', 'SB-112'] Len: 23
Total records in each file: 207


# Execution

In [9]:
TL_Base_Model(100,300, p_ids)

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 11 o' clock and 0 minutes
~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
Before balancing count  1.0    2659
0.0    1021
Name: class, dtype: int64


2023-08-24 11:00:34.915350: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


~~~~~~~ value_counts: ~~~~~~~
SHOULD BE BALANCED NOW: 1.0    2659
0.0    2659
Name: class, dtype: int64


/home/st/sbhuvaji/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

2/2 [==============================] - 0s 2ms/step
cm [[ 7  4]
 [16 33]]
cm.ravel() [ 7  4 16 33]
done testing 1 of 3.
done RE-training 2 of 3. 

/home/st/sbhuvaji/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 2ms/step
cm [[ 4  7]
 [ 8 41]]
cm.ravel() [ 4  7  8 41]
done testing 2 of 3.


100%|██████████| 26/26 [42:43<00:00, 98.60s/it]

evaluation results saved for TLSI_model_100_300


In [10]:
print("Done")

Done


# End